PRE-PROCESSING THE DATA

In [ ]:
# libraries that we're loading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras import layers, models, applications, optimizers
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Loading the data and processing it so that we can transfer it into the model
root_dir = "/content/drive/MyDrive/TechTitans/Brain/Brain/dirty_train"
disease_names = ["glioma", "meningioma", "notumor", "pituitary"]

# Function to convert the images into vectorize form and provide them label
def load_images_and_labels(root_dir, disease_names, img_size=(224,224)):
    # Creating an empty list to store the images and their corresponding labels
    data = []

    # Loading the images from each folder present into the brain folder
    for i, disease_name in enumerate(disease_names):
        # full path to the disease folder
        disease_path = os.path.join(root_dir, disease_name)

        # loading image files in the current disease folder
        image_files = os.listdir(disease_path)

        # getting each image file in the current folder
        for image_file in image_files:

            # Checking image
            if image_file.lower().endswith((".jpg", ".png", ".jpeg")):
                # full path to the image file
                image_path = os.path.join(disease_path, image_file)

                # Loading the image using OpenCV
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Warning: Could not read image: {image_path}")
                    continue    # Skip to the next image if loading fails

                # resizing the image to a consistent size and converting into the grayscale image
                img = cv2.resize(img, img_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                label = i
                data.append((img, label))

    print(f"Total images loaded: {len(data)}")
    return data


In [2]:
# function to remove the noise from the images
def preprocess_mri_image(gray_image):
    # Ensure uint8 format
    if gray_image.dtype != 'uint8':
        gray_image = (gray_image * 255).astype('uint8')

    # Denoising
    denoised_image = cv2.fastNlMeansDenoising(gray_image, None, 4, 7, 21)

    # CLAHE for local contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(6, 6))
    enhanced_image = clahe.apply(denoised_image)

    return enhanced_image

In [3]:
# function to apply remove_noise and split the initial corpus into vectors of image and labels
def preprocess_data(data):
    corpus = []
    labels = []
    for img, label in data:

        # remove noise from the image
        denoised_img = preprocess_mri_image(img)

        # appending the denoised image to the new corpus
        corpus.append(denoised_img)
        labels.append(label)
    return np.array(corpus), np.array(labels)

In [4]:
# Function to prepare the dataset for training the model
def prepare_dataset(images, labels, batch_size=32, shuffle=True):
    # normalizing the images and expand dims for channel
    images = images.astype('float32') / 255.0
    images = np.expand_dims(images, axis=-1)
    labels = to_categorical(labels, num_classes=len(disease_names))
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


CREATING THE MODEL

In [5]:
class BrainMRIClassifierOptimized:
    # constructor to initalise the values
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    # function to buil the model
    def build_cnn(self):
        inputs = layers.Input(shape=self.input_shape)
        x = layers.Conv2D(32, (7, 7), padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)

        # adjust the residual connection to match the number of filters
        residual = layers.Conv2D(64, (1, 1), padding='same')(x)  # Matching the filter size
        residual = layers.BatchNormalization()(residual)

        x = layers.Conv2D(64, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.add([x, residual])  # Adding the residual connection

        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Dense(64 // 16, activation='relu')(se)
        se = layers.Dense(64, activation='sigmoid')(se)
        se = layers.Reshape((1, 1, 64))(se)
        x = layers.multiply([x, se])

        x = layers.Conv2D(128, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.5)(x)

        outputs = layers.Dense(self.num_classes, activation='softmax')(x)

        # compliling the model together with the inputs and outputs layers
        model = models.Model(inputs, outputs)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=1e-4),
            loss='categorical_crossentropy',
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')],
        )
        return model

In [6]:
# loading and preprocess data
data = load_images_and_labels(root_dir, disease_names)
images, labels = preprocess_data(data)


NameError: name 'os' is not defined

In [ ]:
image = plt.imread("/content/drive/MyDrive/TechTitans/Brain/Brain/dirty_train/pituitary/Tr-piTr_0008.jpg")
bgimage = gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
primage = preprocess_mri_image(bgimage)

plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.imshow(bgimage, cmap='gray')
plt.title('Original MRI Slice')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(primage, cmap='gray')
plt.title('Processed MRI Slice')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# dividing the data into train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
# preapring the dataset
batch_size = 32
train_ds = prepare_dataset(X_train, y_train, batch_size=batch_size, shuffle=True)
test_ds = prepare_dataset(X_test, y_test, batch_size=batch_size, shuffle=False)

In [ ]:
# building the model
input_shape = (224, 224, 1)
num_classes = len(disease_names)
classifier = BrainMRIClassifierOptimized(input_shape, num_classes)
model = classifier.build_cnn()

In [ ]:
# training model
history = model.fit(train_ds, epochs=20, validation_data=test_ds)

# evaluateing the model
loss, accuracy, auc = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}, Test AUC: {auc:.4f}")

Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 26s 123ms/step - accuracy: 0.3751 - auc: 0.6367 - loss: 1.3070 - val_accuracy: 0.2441 - val_auc: 0.5295 - val_loss: 1.3952
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.4996 - auc: 0.7478 - loss: 1.1723 - val_accuracy: 0.2319 - val_auc: 0.5677 - val_loss: 1.4564
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 93ms/step - accuracy: 0.5494 - auc: 0.7970 - loss: 1.0832 - val_accuracy: 0.2502 - val_auc: 0.6106 - val_loss: 1.5379
Epoch 4/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 92ms/step - accuracy: 0.5671 - auc: 0.8125 - loss: 1.0398 - val_accuracy: 0.3330 - val_auc: 0.6622 - val_loss: 1.3703
Epoch 5/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 20s 91ms/step - accuracy: 0.5913 - auc: 0.8333 - loss: 0.9937 - val_accuracy: 0.4690 - val_auc: 0.7620 - val_loss: 1.1394
Epoch 6/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.6265 - auc: 0.8513 - loss: 0.9472 - val_accuracy: 0.6129 - val_auc: 0.8524 - val_loss: 0.9344
Epoch 7/20
144/144 ━━━━━━━━

In [ ]:
# saving the model so taht we can use it to use in backend
model.save('/content/drive/MyDrive/TechTitans/my_model.keras')